# Compare Emma's New Data to Old Data

- turns out using magleft etc was not correct ** 
- needed to use "data" 

In [86]:
import scipy.io
import pandas as pd
import numpy as np
import glob
import pickle
import os
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
%matplotlib inline

## Exploring her mat file

In [87]:
dictt = scipy.io.loadmat('../data/data_emmas_1_15_18/DataMatrix_AllfMRI_FinalSubjects.mat')

/Users/chris/anaconda/lib/python3.6/site-packages/scipy/io/matlab/mio.py:136: MatReadWarning: Duplicate variable name "None" in stream - replacing previous with new
Consider mio5.varmats_from_mat to split file into single variable files
  matfile_dict = MR.get_variables(variable_names)


In [88]:
dictt['data'][0][0][:,5] #amb
dictt['data'][0][0][:,3] # ml
dictt['data'][0][0][:,4] # mr
dictt['data'][0][0][:,0] # p
dictt['data'][0][0][1:10,1] # p 2 is beta-corrected I believe. 

array([ 0.54,  0.12,  0.12,  0.28,  0.08,  0.36,  0.56,  0.42,  0.56])

In [89]:
dictt['data'][0][0][1:10,2]

array([ 0.54 ,  0.15 ,  0.125,  0.28 ,  0.1  ,  0.36 ,  1.   ,  0.5  ,
        0.56 ])

In [90]:
#dictt['data'][sub][0][:,2]*dictt['data'][sub][0][:,5]*50

In [91]:
#dictt['data'][0][sub]
amb=(dictt['data'][0][0][:,5]!=1.0)
dictt['data'][0][0][amb,2][1:10] # 3 that she uses. This is revealed

array([ 0.125,  0.1  ,  1.   ,  0.5  ,  0.4  ,  0.   ,  1.   ,  0.5  ,  0.   ])

In [92]:
#dictt['data'][0][sub]
dictt['data'][0][0][amb,1][1:10] # 2 this must be true prob.. not revealed.. otherwise no way 56-->1.0

array([ 0.12,  0.08,  0.56,  0.42,  0.36,  0.12,  0.4 ,  0.62,  0.16])

In [93]:
dictt['pShock'][1:10,0] # beta corrected. 

array([ 0.14285571,  0.166665  ,  0.66667   ,  0.499995  ,  0.42856714,
        0.0833325 ,  0.66667   ,  0.499995  ,  0.199998  ])

In [94]:
dictt['data'][0][sub][:,2]*dictt['data'][0][sub][:,5]*50.0

array([ 12.,  27.,   3.,   5.,  14.,   1.,  18.,   1.,   5.,  28.,   2.,
         0.,  38.,   1.,   1.,  12.,   0.,   2.,  46.,   4.,   5.,  14.,
         3.,   4.,   1.,  24.,   1.,   7.,  26.,  14.,  10.,  10.,  12.,
        34.,   2.,  36.,   1.,  20.,   1.,   3.,   0.,  33.,   0.,  38.,
         0.,   3.,  23.,  14.,  30.,   0.,  38.,   6.,  37.,   1.,  42.,
        36.,  32.,   0.,   1.,  36.,  16.,  20.,  12.,  44.,  30.,   1.,
         1.,  40.,   1.,   1.,  42.,   9.,  32.,  11.,   0.,  24.,  20.,
         0.,  34.,  15.,   2.,   1.,  39.,  26.,   1.,  46.,  28.,  14.,
         5.,   2.,  46.,  27.,   3.,  23.,  10.,  14.,   2.,   1.,   4.,
        40.,  48.,   2.,  25.,  19.,   1.,  42.,   1.,   3.,   1.,  39.,
        16.,  32.,   2.,  42.,   2.,  20.,  25.,   2.,   7.,   9.,  32.,
         8.,   4.,   2.,  21.,  34.,  10.,  44.,   0.,  38.,   2.,   8.,
        45.,  30.,   3.,  24.,  30.,  32.,   1.,  20.,  19.,   1.,  31.,
         8.,  36.,  15.,   1.,   1.,  26.,  28.,  3

## Saving out Data

In [95]:
# convert data into a form that is useable for my model scripts. 
MIDs = []
for sub in range(32):
    trial_array = np.empty((200,8))
    
    # basic 
    trial_array[:,0] = dictt['choices'][:,sub] # check this is right/left 
    trial_array[:,1] =dictt['data'][0][sub][:,0] # p
    trial_array[:,2] =dictt['data'][0][sub][:,2] # revealed probl 
    trial_array[:,3] =dictt['data'][0][sub][:,3]/100.0 # ml
    trial_array[:,4] =dictt['data'][0][sub][:,4]/100.0 # mr
    trial_array[:,5] =dictt['data'][0][sub][:,5] #amb
    trial_array[:,6] =dictt['data'][0][sub][:,5]*50.0 # revealed number 
    trial_array[:,7] =dictt['data'][0][sub][:,2]*dictt['data'][0][sub][:,5]*50.0 # calculating revealed o 
    
    
    # make dataFrame
    trial_data = pd.DataFrame(data=trial_array,columns=['resp_r_1','prob_x_l','prob_x_r','mag_left','mag_right',
                                                        'info_amb','num_revealed','revealed_x_r'])
    
    # response 
    trial_data.loc[trial_data['resp_r_1']>1,'resp_r_1']=np.nan
    trial_data['resp_amb_1'] = trial_data['resp_r_1']
    
    # mag 
    trial_data['mag_ambig']=trial_data['mag_right']
    trial_data['mag_unambig']=trial_data['mag_left']
    
    # prob
    # changed this to say 'x' instead of 'o'. (not substantive change)
    # for the scripts that hanna wrote
    trial_data['prob_x_unambig']=trial_data['prob_x_l']
    trial_data['prob_x_ambig']=trial_data['prob_x_r']

    trial_data['prob_o_l']=1.0-trial_data['prob_x_l']
    trial_data['prob_o_r']=1.0-trial_data['prob_x_r']
    
    # actual number of revealed tokens. (NOT SURE IF I HAVE YET)
    trial_data['revealed_x_ambig']=trial_data['revealed_x_r'] # right side is ambig
    trial_data['revealed_o_ambig']=trial_data['num_revealed']-trial_data['revealed_x_ambig']
    
    # beta correct
    
    trial_data.loc[trial_data['info_amb']!=1.0,'prob_x_ambig_bayes_emmas'] = dictt['pShock'][:,sub]
    trial_data['prob_x_ambig_bayes_mine']=(trial_data['revealed_x_ambig']+1)/(trial_data['num_revealed']+2)
    trial_data['prob_x_ambig_bayes']=trial_data['prob_x_ambig_bayes_mine'].copy()
    
    # ambiguity level 
    trial_data['info_amb_sqrt']=np.sqrt(trial_data['info_amb'])
    trial_data['ambig_r']=(trial_data['info_amb_sqrt']!=1).astype('int')
    trial_data['ambig_l']=np.zeros(len(trial_data))
    trial_data['ambiguityLevel'] = 1.0 - trial_data['info_amb_sqrt']#np.sqrt(trial_data['revealed_o_r'])
    trial_data['revealed_ambi']=trial_data['info_amb_sqrt']
    
    trial_data['gain_or_loss_trial']='gain'
    
    trial_data['MID']='S'+str(sub)
    MIDs.append('S'+str(sub))
    
    trial_data.to_csv('../data/data_emmas_logfiles_from_mat_1_15_17/S'+str(sub)+'.csv')




- my beta-corrected prob matches hers. 

In [96]:
trial_data['prob_x_ambig_bayes'][1:10]

1    0.538462
2    0.181818
3    0.142857
4    0.288462
5    0.166667
6    0.365385
7    0.666667
8    0.500000
9    0.557692
Name: prob_x_ambig_bayes, dtype: float64

In [97]:
trial_data['prob_x_ambig_bayes_emmas'][1:10]

1         NaN
2    0.181816
3    0.142856
4         NaN
5    0.166665
6         NaN
7    0.666670
8    0.499995
9         NaN
Name: prob_x_ambig_bayes_emmas, dtype: float64